## Búsqueda de ubicación óptima para instalaciones

### Importación de librerías y módulos

In [1]:
from pymongo import MongoClient
from src.coordenadas import *
from src.cleaning import *
from src.airports import *
from src.representation import *
from src.init_tokens import *
import random
import pandas as pd

import geopandas as gpd
from cartoframes.viz import Map, Layer
from cartoframes.viz.helpers import size_continuous_layer
from cartoframes.viz.widgets import histogram_widget

### Necesidades

**Diseñadores** ➡️ Sitio donde hablen sobre diseño + Compañías de diseño

**Empleados con hijos** ➡️ Guardería

**Desarrolladores** ➡️ Empresas de tecnología de éxito

**Ejecutivos** ➡️ Establecimiento de Starbucks

**Contables** ➡️ Aeropuerto

**Consenso** ➡️ Empresas < 10 años

**Empleados jóvenes** ➡️ Sitios de fiesta

**CEO** ➡️ Restaurantes veganos


### Elección de prioridades

**Modelo aleatorio para narrativa**

In [2]:
necesidades = ["Diseñadores","Empleados con hijos","Desarrolladores","Ejecutivos","Contables","Empleados jóvenes","Todos","CEO"]

random_priorities = list(random.sample(range(1,len(necesidades)+1),len(necesidades)))

prioridades = sorted(list(map(lambda x: (random_priorities[necesidades.index(x)],x),necesidades)))[:3]

prioridades


[(1, 'Todos'), (2, 'Diseñadores'), (3, 'Empleados con hijos')]

### Importación de dataset principal

In [14]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost/AnalysisPlaces")

db = client.get_database()

db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'AnalysisPlaces')

**Corrección de coordenadas**

In [4]:

officesCoord = cleaning(db["companiesOriginal"])

# officesCoord.to_json("Outputs/companiesCoord.json", orient="records")


,name,category_code,location
0,Digg,news,"{'type': 'Point', 'coordinates': [-122.394523,..."
1,Slacker,music,"{'type': 'Point', 'coordinates': [-117.081406,..."
2,AdventNet,enterprise,"{'type': 'Point', 'coordinates': [-121.904945,..."


**Creación de DataFrame**

In [15]:
filter = {"$and":[{"location":{"$not":{"$size":0}}},{"location":{"$ne":None}},{"location":{"$exists":True}}]}
project={'name': 1, "category_code":1, "location":1, '_id':0}

test = pd.DataFrame(list(db['companiesCoord'].find(filter=filter,projection=project)))

test.head(3)

,name,category_code,location
0,Digg,news,"{'type': 'Point', 'coordinates': [-122.394523,..."
1,Slacker,music,"{'type': 'Point', 'coordinates': [-117.081406,..."
2,AdventNet,enterprise,"{'type': 'Point', 'coordinates': [-121.904945,..."


**Creamos columna de localización principal**

In [5]:
dfCoords = test[["location"]].apply(lambda r: r.location, result_type="expand", axis=1)

dfCoords.head(3)

,type,coordinates
0,Point,"[-122.394523, 37.764726]"
1,Point,"[-117.081406, 33.022176]"
2,Point,"[-121.904945, 37.692934]"


**Creamos columnas de localización adicionales**

In [6]:
test["long"] = dfCoords["coordinates"].apply(lambda x: x[0])
test["lat"] = dfCoords["coordinates"].apply(lambda x: x[1])

test = test[['name', 'category_code', 'long', 'lat', 'location']] 

test.head(3)

,name,category_code,long,lat,location
0,Digg,news,-122.394523,37.764726,"{'type': 'Point', 'coordinates': [-122.394523,..."
1,Slacker,music,-117.081406,33.022176,"{'type': 'Point', 'coordinates': [-117.081406,..."
2,AdventNet,enterprise,-121.904945,37.692934,"{'type': 'Point', 'coordinates': [-121.904945,..."


### Necesidad 1 ➡️ Starbucks

In [7]:
starbucksLocation = pd.read_csv("Inputs/Starbucks_Locations.csv")

starbucksLocation.head(3)

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39


In [8]:
starbucksLocation = starbucksLocation[["Store Name", "Latitude", "Longitude"]].rename(columns={"Latitude":"long","Longitude":"lat"})

starbucksCoords = geojsonize(starbucksLocation)

starbucksCoords.head(3)

,Store Name,long,lat,location
0,"Meritxell, 96",42.51,1.53,"{'type': 'Point', 'coordinates': [42.51, 1.53]}"
1,Ajman Drive Thru,25.42,55.47,"{'type': 'Point', 'coordinates': [25.42, 55.47]}"
2,Dana Mall,25.39,55.47,"{'type': 'Point', 'coordinates': [25.39, 55.47]}"


**Transformación a representación gráfica**

In [9]:
Map(Layer(starbucksCoords.iloc[0:130]))

/home/alex/.local/lib/python3.6/site-packages/cartoframes/utils/geom_utils.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geometry'] = _compute_geometry_from_latlng(lat_column, lng_column)


### Necesidad 2 ➡️ Aeropuertos 

In [10]:
airportsCoord = airportsDF()

airportsCoord.head()

,nombre,long,lat
0,Aleknagik / New Airport,-158.617996,59.282600
1,Honiara International Airport,160.054993,-9.428000
2,Munda Airport,157.263000,-8.327970
3,Mar de Cortés International Airport,-113.305864,31.351621
4,Nauru International Airport,166.919006,-0.547458


**Transformación a representación gráfica**

In [11]:
airportsMap = geojsonize(airportsCoord)

airportsMap.head(3)

,nombre,long,lat,location
0,Aleknagik / New Airport,-158.617996,59.28260,"{'type': 'Point', 'coordinates': [-158.6179962..."
1,Honiara International Airport,160.054993,-9.42800,"{'type': 'Point', 'coordinates': [160.05499267..."
2,Munda Airport,157.263000,-8.32797,"{'type': 'Point', 'coordinates': [157.26300048..."


In [12]:
Map(Layer(airportsMap))

### Necesidad 3 ➡️ Guarderías

In [13]:
#IT WORKS!!

init_four(lat, long)

NameError: name 'lat' is not defined